# チームの成績取得
- チームの成績を取得する。
- 年間の順位表を取得する。

In [1]:
import requests
import lxml.html
import pandas as pd
import numpy as np
import time
import sqlite3
import re

In [3]:
# 過去の成績の一覧ページから、成績表を取得する。
response = requests.get('http://npb.jp/bis/yearly/')
root = lxml.html.fromstring(response.content)
root.make_links_absolute(response.url)
# ○○年になっているところに目的のページに行くためのURLが埋め込まれている。
years = root.cssselect('.yearlySeason > a')

scores = []
# 年ごとの勝敗表を取得する。
for i, year in enumerate(years):
    # 野球連盟のページは取得しない。
    if not re.search('yakyuremmei', year.get('href')):
        df_score = get_leag_score(year.get('href'))
        scores.append(df_score)
    
#     if i==10:
#         break
# データを結合する。
win_loss_table = pd.concat(scores, axis=0, ignore_index=True)

# クロールした結果をデータベースに保存する。
conn = sqlite3.connect('players_info.db')
c = conn.cursor()
win_loss_table.to_sql('win_loss_table', conn, if_exists='replace')

conn.commit()
conn.close()

/home/kyohei/anaconda2/envs/py36con/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [2]:
def get_leag_score(url):
    # URLにセントラルかパシフィックかを分ける情報と、何年かの情報が含まれている
    # reで判定
    time.sleep(1)
    if re.search('centralleague', url):
        leag_name = 'セントラルリーグ'
        year = re.search('(\d+)', url).group(1)
        df = pd.io.html.read_html(url)
    else:
        leag_name = 'パシフィックリーグ'
        year = re.search('(\d+)', url).group(1)
        df = pd.io.html.read_html(url)
    # その年のチーム数を取得する。
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    teams = root.cssselect('.contentsPadding')
    team_no = int(len(teams[0].cssselect('.yearlyMain')) / 2)  # アイテムの個数割る2
    
    # DataFrameで取得した情報は勝敗表以外の情報も含まれているので、
    # 勝敗表の情報に整形する。
    # リーグ戦の情報に限定
    df_ = df[2].iloc[1:, :8]
    # 列名を1行名の値に変更
    df_.columns = df_.iloc[0]
    # 必要な行に限定して、np.nanの列を削除
    df_2 = df_.iloc[1:team_no+1].dropna(axis=1)
    # indexを使って順位をつける
    df_3 = df_2.reset_index()
    df_3['順位'] = df_3['index'] - 1
    df_3.drop('index', axis=1, inplace=True)
    # 年とリーグ名を追加する
    df_3['年'] = year
    df_3['リーグ'] = leag_name
    df_3['URL'] = url
    
    return df_3